<a href="https://colab.research.google.com/github/BRIAN12682/Automation-Projects/blob/main/Imagetotext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio transformers pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 12.6 MB/

## CLIP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel, GPT2Tokenizer, GPT2LMHeadModel

# Load models and tokenizer
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Function to get image description using CLIP
def get_image_description(image_path, candidate_texts):
    image = Image.open(image_path)
    inputs = clip_processor(images=image, return_tensors="pt")
    image_features = clip_model.get_image_features(**inputs)
    image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)

    text_inputs = clip_processor(text=candidate_texts, return_tensors="pt", padding=True)
    text_features = clip_model.get_text_features(**text_inputs)
    text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)

    # Calculate similarity
    similarities = torch.matmul(image_features, text_features.T)
    best_match_idx = similarities.argmax().item()
    return candidate_texts[best_match_idx]

# Function to generate detailed explanation using GPT-2
def generate_explanation(description):
    inputs = gpt_tokenizer.encode(description, return_tensors="pt")
    attention_mask = torch.ones_like(inputs)

    outputs = gpt_model.generate(
        inputs,
        max_length=50,
        num_return_sequences=1,
        pad_token_id=gpt_tokenizer.eos_token_id,
        attention_mask=attention_mask,
        top_k=50,  # Using top-k sampling to add randomness
        temperature=0.7,  # Adjust temperature for more diverse results
        do_sample=True,  # Enable sampling
        top_p=0.9  # Nucleus sampling
    )

    explanation = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return explanation

# Example usage
image_path = "/content/drive/MyDrive/ProblemTree2.jpg"
candidate_texts = [
    "a picture of a tree", "a picture of a person", "a picture of a car",
    "a picture of a building", "a picture of a dog"
]

description = get_image_description(image_path, candidate_texts)
print("Description:", description)
explanation = generate_explanation(description)
print("Explanation:", explanation)


Description: a picture of a person
Explanation: a picture of a person who had been arrested for sexual assault.

The charges are being brought by the victim's father, who said he was assaulted while he was at his job.

"I am so ashamed of what happened to my


## DALL-E

In [ ]:
pip install --upgrade openai

In [ ]:
# Ensure you have the necessary packages installed
!pip install openai pillow

from PIL import Image
import openai
import io

# Initialize OpenAI API
openai.api_key = 'your-openai-api-key'

# Load an image
image_path = "/content/drive/MyDrive/ProblemTree2.jpg"
image = Image.open(image_path)

# Function to generate detailed explanation using GPT-3/GPT-4
def generate_explanation_from_image(image_path):
    # Load the image
    with open(image_path, 'rb') as image_file:
        image_bytes = image_file.read()

    # You might need to upload this image to a server or use another method to process it
    # Here, we will describe it assuming we could have an API process the image.

    # For now, let's assume the image description is:
    description = "a picture of a tree"

    # Generate explanation
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=f"Describe the following image in detail: {description}",
        max_tokens=50,
        temperature=0.7
    )

    explanation = response.choices[0].text.strip()
    return description, explanation

# Example usage
description, explanation = generate_explanation_from_image(image_path)
print("Description:", description)
print("Explanation:", explanation)


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


# New Section